___
<img style="float: right; margin: 0px 0px 15px 15px;" src="https://machinelearningmastery.com/wp-content/uploads/2016/03/Compare-Machine-Learning-Algorithms.png" width="350px" height="180px" />


# <font color= #8A0829> Laboratorio de Modelado de Datos </font>
#### <font color= #2E9AFE> `Martes y Viernes (Videoconferencia) de 13:00 - 15:00 hrs`</font>
- <Strong> Sara Eugenia Rodríguez </Strong>
- <Strong> Año </Strong>: 2023
- <Strong> Email: </Strong>  <font color="blue"> `cd682324@iteso.mx` </font>
___


### <font color= #2E9AFE> Tema: Comparación de modelos de regresión</font>

Una vez que aprendimos a crear los modelos de machine learning ¿qué hacemos con ellos?

Comparar varios modelos de machine learning para problemas de regresión es necesario para poder encontrar cuál de todos los modelos es el más eficiente y tiene los resultados más precisos. 

Hay muchos criterios para comparar los modelos. 

Recordando que si la **variable de respuesta es continua** entonces, este es un problema de **regresión** y tenemos que usar modelos de regresión para estimar los valores predichos. 

Como vimos, hay muchos modelos de regresión candidatos. Nuestra tarea es encontrar el que sirva a nuestro propósito.

Vamos a comparar los siguientes modelos:
- Regresión lineal múltiple
- Regresión Ridge
- Árbol de decisión
- Bosques Aleatorios
- SVR
- Redes Neuronales


Vamos a utilizar varias métricas para comparar los modelos.

### Métricas de Performance (Regresión)

**Error Absoluto Medio (Mean Absolute Error - MAE)**

Si $y$ es la variable de respuesta y $\hat{y}$ es la predicción, entonces el MAE es el error entre estos pares ($y$,$\hat{y}$) de variables y se calcula de la forma:

$$MAE = \frac{\sum_{i=1}^{n}|y_{i}-\hat{y}_{i}|}{n}$$

*¿Qué hace?*
- Mide el promedio del error absoluto entre los valores reales y las predicciones

*Ventajas*
- Es una métrica independiente de la escala, lo que significa que está en las mismas unidades de las variables originales. 
- Fácil de interpretar
- Es buena opción si no nos importan los valores atípicos

*Desventajas*
- Se les da la misma importancia (peso) a todos los errores
- Es insensible a valores atípicos



**Error Cuadrático Medio (Mean Square Error - MSE)**


$$MSE = \frac{\sum_{i=1}^{n}(y_{i}-\hat{y}_{i})^{2}}{n}$$

*¿Qué hace?*
El MSE calcula el promedio de los cuadrados del error entre los valores verdaderos y los valores estimados

*Ventajas*
- Función diferenciable
- El efecto de errores más grandes se vuelve más pronunciado que los errores más pequeños

*Desventajas*
- Tiene diferentes unidades que los valores reales


**Diferencia entre el MAE y el MSE**

La gran diferencia es cómo responden a errores más grandes. 

Depende mucho de la decisión del negocio y el costo asociado a los errores. 
     
 **R^2**
 
 $$R^{2}=1-\frac{SS_{res}}{SS_{Tot}}= \frac{\sum_{i=1}^{n}(y_{i}-\hat{y}_{i})^{2}}{\sum_{i=1}^{n}(y_{i}-\bar{y}_{i})^{2}}$$
 
 *¿Qué hace?*
- Explica la proporción de la varianza de la salida ("Y") que puede ser explicada por los predictores ("X")

*Desventajas*
- Siempre incrementa añadiendo más variables independientes (X), lo cual es una desventaja porque esas variables pueden no ser importantes para el modelo

*Ventajas*
- Es una métrica relativa que se usa para comparar varios modelos que fueron entrenados con los mismos datos

     
     
 

### Los Datos

Tenemos un dataset de personas que tienen un carro y lo quieren vender. Se tienen las siguientes variables:

- Age 
- Gender
- Miles: promedio de millas manejadas por día
- Debt
- Income
- **Sales**

Basados en esas variables, queremos predecir el valor potencial de venta de un carro, por lo que variable a predecir es "Sales". 

In [1]:
#Librerías
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

#Selección de variables
from sklearn.feature_selection import RFE

#regresión lineal
from sklearn.linear_model import LinearRegression
#Ridge
from sklearn.linear_model import Ridge
#árbol de decisión
from sklearn.tree import DecisionTreeRegressor
#Bosque aleatorio
from sklearn.ensemble import RandomForestRegressor
#Maquina de Vector Soporte (SVR)
from sklearn.svm import SVR
#Redes Neuronales
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from scikeras.wrappers import KerasRegressor
from keras.optimizers import SGD, Adam
#XGboost
import xgboost as xgb

#métricas de performance
from sklearn.metrics import mean_squared_error, make_scorer, mean_absolute_error, r2_score

#Cross validation y train-test split
from sklearn.model_selection import train_test_split,cross_val_score, cross_val_predict, KFold

#Grid Search
from sklearn.model_selection import GridSearchCV

In [2]:
#cargar datos
datos = pd.read_csv('carros.csv')
datos.head()

,age,gender,miles,debt,income,sales
0,28,0,23,0,4099,620
1,26,0,27,0,2677,1792
2,30,1,58,41576,6215,27754
3,26,1,25,43172,7626,28256
4,20,1,17,6979,8071,4438


In [3]:
#tamaño de los datos
datos.shape

(963, 6)

## Exploración de Datos

In [4]:
#Revisamos si hay datos nulos
missing = pd.DataFrame(datos.isnull().sum(),columns=['Valores_Nulos'])
missing

,Valores_Nulos
age,0
gender,0
miles,0
debt,0
income,0
sales,0


In [5]:
#Creamos gráficos básicos de los datos
sns.pairplot(datos[['age', 'miles', 'debt', 'income', 'sales']], diag_kind="kde")

In [6]:
# Calculo de la asimetria con pandas
skewness = round(datos.skew(),2)
skewness = skewness.to_frame()
skewness = skewness.rename(columns={0: "value"}) 

def f(x):
    if x['value'] < -1 or x['value'] > 1: return 'Highly Skewed'
    elif (x['value']<=0 and x['value']>=-0.5) or (x['value'] >=0 and x['value']<=0.5):
        return 'Symmetric distribution'
    else: return 'Moderately skewed'
    
skewness['skewness'] = skewness.apply(f, axis=1)

skewness

,value,skewness
age,0.20,Symmetric distribution
gender,-0.05,Symmetric distribution
miles,2.18,Highly Skewed
debt,1.34,Highly Skewed
income,-0.11,Symmetric distribution
sales,0.49,Symmetric distribution


Los datos no están tan sesgados, por lo que podemos dejarlos de esta manera y no transformar los atípicos. 

In [7]:
#Calculando estadísticas básicas con los datos
datos_stats = datos.describe()
datos_stats = datos_stats.transpose()
datos_stats

,count,mean,std,min,25%,50%,75%,max
age,963.0,37.971963,12.290838,19.0,27.0,37.0,49.0,60.0
gender,963.0,0.512980,0.500091,0.0,0.0,1.0,1.0,1.0
miles,963.0,27.704050,13.378181,10.0,20.0,25.0,32.0,97.0
debt,963.0,14109.004154,18273.702481,0.0,1475.0,6236.0,16686.0,59770.0
income,963.0,6176.047767,3260.670142,0.0,3506.5,6360.0,8649.5,11970.0
sales,963.0,11689.860852,8986.896921,500.0,3554.0,9130.0,19245.0,29926.0


De las estadísticas anteriores podemos observar que algunas variables tienen grandes rangos y desviaciones, lo cual puede crear problemas durante el ajuste del modelo. Por lo tanto sería buena idea escalar los datos. 


### Cross Validation con optimización de hiperparámetros

Por lo general, cuando todavia no sabemos qué modelo es el mejor para nuestros datos, cross validation nos puede ayudar a hacer una comparación más justa entre diferentes modelos. 

La idea es:
1. dividir los datos en train y test
2. entrenar múltiples modelos muestreando los datos de entrenamiento. 
3. Probar el modelo en el test set
4. Seleccionar el modelo con nos dio la mejor calificación
5. Hacer predicciones con el modelo ganador




<img style="float: center; margin: 0px 0px 15px 15px;" src="https://scikit-learn.org/stable/_images/grid_search_cross_validation.png" width="450px" height="280px" />



In [8]:
#separar la X y Y
X=datos.iloc[:,0:5]
y=datos.iloc[:,5]

#Dividimos los datos en train y test para después hacer el cross validation sólo con los datos del train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform (X_test)

**¿Porqué estandarizar los datos después del train test split?**

El escalamiento debe realizarse después de dividir los datos en entrenamiento y prueba, y se debe hacer sólo usando los datos de entrenamiento. 

Esto se debe a que los datos de prueba (test) tienen el papel de datos no vistos por el modelo, por lo que se supone que estos datos no están accesibles en la estapa del entrenamiento. 

Usar cualquier información proveniente de los datos de prueba antes o durante el entranamiento puede afectar al desempeño del modelo

In [10]:
#Configurar los folds que vamos a usar para todos los modelos
n_folds = 5
seed = 7
scoring = 'neg_root_mean_squared_error' #usamos el error cuadrático medio como medida de performance
kfold = KFold(n_splits=n_folds, random_state=seed, shuffle=True)

In [11]:
#métricas que vamos a ver
scoring = ['r2', 'neg_mean_absolute_error','neg_root_mean_squared_error']

#### Modelo 1: Regresión Lineal

In [12]:
#Inicializar modelo
lm=LinearRegression()

#Hacer el cross validation y probar con el train
r2_lr= (cross_val_score(lm, X_train,y_train, cv=kfold,  scoring='r2')).mean()
mae_lr= -(cross_val_score(lm, X_train,y_train, cv=kfold,  scoring='neg_mean_absolute_error')).mean()
mse_lr= (-cross_val_score(lm, X_train,y_train, cv=kfold,  scoring='neg_root_mean_squared_error')).mean()

#Performance en el train
print("R2 train", r2_lr)
print("MAE Train", mae_lr)
print("RMSE Train", mse_lr)

R2 train 0.8158013380172221
MAE Train 2915.6412524254692
RMSE Train 3801.2012383692795


In [13]:
#Performance en el test
lm.fit(X_train,y_train)
y_pred = lm.predict(X_test)

mae_lr=mean_absolute_error(y_test,y_pred)
rmse_lr=np.sqrt(mean_squared_error(y_test,y_pred))
r2_lr=r2_score(y_test,y_pred)

results_lr = pd.DataFrame([['Linear regression', mae_lr,rmse_lr,r2_lr]],columns=['Model', 'MAE', 'RMSE', 'R2'])
results_lr

,Model,MAE,RMSE,R2
0,Linear regression,2842.934786,3666.939818,0.836061


#### Modelo 2: Regresión lineal con regularización Rigde

Aquí vamos a usar RidgeCV para encontrar el valor óptimo de $\lambda$
Una vez encontrado ese valor óptimo, creamos el modelo con el valor de $\lambda$ óptimo

In [14]:
from sklearn.linear_model import RidgeCV

#Inicializar modelo de Ridge con cross validation para seleccionar el hiperparámetro de alpha óptimo
#Probamos con 3 valores diferentes de alpha a ver cuál es el mejor
model = RidgeCV(alphas=np.arange(0.01,5,0.01), cv=kfold, scoring='r2') 

#Entreno modelo
model.fit(X_train, y_train)

#¿Cuál es el alfa óptima?
print('alfa: %f' % model.alpha_)


alfa: 4.990000


In [15]:
#Creo el modelo con el alpha óptima
model_ridge_new=Ridge(alpha=4.99)
#Entrenar el nuevo modelo con la alpha óptima
model_ridge_new.fit(X_train, y_train)

#Performance en el test
y_pred = model.predict(X_test)

mae_ridge=mean_absolute_error(y_test,y_pred)
rmse_ridge=np.sqrt(mean_squared_error(y_test,y_pred))
r2_ridge=r2_score(y_test,y_pred)

results_ridge = pd.DataFrame([['Linear regression Ridge', mae_ridge,rmse_ridge,r2_ridge]],columns=['Model', 'MAE', 'RMSE', 'R2'])
results_ridge

,Model,MAE,RMSE,R2
0,Linear regression Ridge,2844.436399,3666.979515,0.836057


#### Modelo 3: Random Forest

Para este modelo vamos a usar cross validation + grid search para encontrar los siguientes hiperparámetros óptimos:
- max_depth (profundidad)
- min_samples_split (número mínimo de observaciones por split)
- n_estimators (número de árboles)


In [16]:
from sklearn.ensemble import RandomForestRegressor

#Inicializo modelo
model = RandomForestRegressor(min_samples_leaf=2,
                               bootstrap=True,
                               oob_score=False,
                               random_state=0,
                               verbose=0)

#Grid search para optimizar hiperparámetros
gs = GridSearchCV(model,
                  param_grid = {'max_depth': range(1, 11), #profundidad del árbol
                                'min_samples_split': range(10, 60, 10), #mínimo numero de observaciones
                                'n_estimators': [100, 300, 500, 800] #número de árboles
                                }, 
                  cv=kfold,
                  scoring='neg_root_mean_squared_error',
                  verbose=0)

#Entreno el modelo
gs.fit(X_train, y_train)


GridSearchCV(cv=KFold(n_splits=5, random_state=7, shuffle=True),
             estimator=RandomForestRegressor(min_samples_leaf=2,
                                             random_state=0),
             param_grid={'max_depth': range(1, 11),
                         'min_samples_split': range(10, 60, 10),
                         'n_estimators': [100, 300, 500, 800]},
             scoring='neg_root_mean_squared_error')

In [17]:
#¿cuáles son los hiperparámetros óptimos?
print(gs.best_params_)

{'max_depth': 8, 'min_samples_split': 30, 'n_estimators': 800}


In [18]:
#crear nuevo modelo usando los parámetros óptimos que obtuvimos
new_model_RF = RandomForestRegressor(n_estimators=800,#número de árboles
                               criterion='squared_error',
                               max_depth=8,
                               min_samples_split=30,
                               min_samples_leaf=2,
                               bootstrap=True,
                               oob_score=False,
                               random_state=0,
                               verbose=0)

#Entreno modelo optimizado
new_model_RF.fit(X_train, y_train)

#Hacer el cross validation y probar con el train
r2_rf= (cross_val_score(new_model_RF, X_train,y_train, cv=kfold,  scoring='r2')).mean()
mae_rf= -(cross_val_score(new_model_RF, X_train,y_train, cv=kfold,  scoring='neg_mean_absolute_error')).mean()
mse_rf= (-cross_val_score(new_model_RF, X_train,y_train, cv=kfold,  scoring='neg_root_mean_squared_error')).mean()

#Performance en el train
print("R2 train", r2_rf)
print("MAE Train", mae_rf)
print("RMSE Train", mse_rf)

R2 train 0.892588214294529
MAE Train 1965.9151891102506
RMSE Train 2897.108349645448


In [19]:
#Performance en el test
y_pred = new_model_RF.predict(X_test)

mae_rf=mean_absolute_error(y_test,y_pred)
rmse_rf=np.sqrt(mean_squared_error(y_test,y_pred))
r2_rf=r2_score(y_test,y_pred)

results_rf = pd.DataFrame([['Random Forest', mae_rf,rmse_rf,r2_rf]],columns=['Model', 'MAE', 'RMSE', 'R2'])
results_rf

,Model,MAE,RMSE,R2
0,Random Forest,1836.106102,2561.377387,0.920012


#### Árbol de decisión
Para este modelo vamos a usar cross validation + grid search para encontrar los siguientes hiperparámetros óptimos:

- max_depth (profundidad)
- min_samples_split (número mínimo de observaciones por split)

In [20]:
#Usando cross validation y grid search
tree = DecisionTreeRegressor()

#Grid search para optimizar hiperparámetros
gs = GridSearchCV(tree,
                  param_grid = {'max_depth': range(1, 11),
                                'min_samples_split': range(10, 60, 10)},
                  cv=kfold,
                  scoring='neg_root_mean_squared_error')

#entreno el modelo
gs.fit(X_train, y_train)



GridSearchCV(cv=KFold(n_splits=5, random_state=7, shuffle=True),
             estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': range(1, 11),
                         'min_samples_split': range(10, 60, 10)},
             scoring='neg_root_mean_squared_error')

In [21]:
#¿cuáles son los hiperparámetros óptimos?
print(gs.best_params_)

{'max_depth': 7, 'min_samples_split': 50}


In [22]:
#crear modelo usando parámetros óptimos
new_model_tree = DecisionTreeRegressor(max_depth=7,
                                  min_samples_split=50)

#Entreno modelo optimizado
new_model_tree.fit(X_train, y_train)

#Hacer el cross validation y probar con el train
r2_tree= (cross_val_score(new_model_tree, X_train,y_train, cv=kfold,  scoring='r2')).mean()
mae_tree= -(cross_val_score(new_model_tree, X_train,y_train, cv=kfold,  scoring='neg_mean_absolute_error')).mean()
rmse_tree= (-cross_val_score(new_model_tree, X_train,y_train, cv=kfold,  scoring='neg_root_mean_squared_error')).mean()

#Performance en el train
print("R2 train", r2_tree)
print("MAE Train", mae_tree)
print("MSE Train", rmse_tree)

R2 train 0.8833332245009938
MAE Train 2015.0780021702842
MSE Train 3019.23963196914


In [23]:
#Performance en el test
y_pred = new_model_tree.predict(X_test)

mae_tree=mean_absolute_error(y_test,y_pred)
rmse_tree=np.sqrt(mean_squared_error(y_test,y_pred))
r2_tree=r2_score(y_test,y_pred)

results_tree = pd.DataFrame([['Decision Tree', mae_tree,rmse_tree,r2_tree]],columns=['Model', 'MAE', 'RMSE', 'R2'])
results_tree 

,Model,MAE,RMSE,R2
0,Decision Tree,1960.984898,2880.276684,0.898855


#### Máquina de vector soporte (SVR)

Para este modelo vamos a usar cross validation + grid search para encontrar los siguientes hiperparámetros óptimos:

- Kernel 
- Hiperparámetros dentro del Kernel
- C

In [24]:
#proponer malla para el gridsearch
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

#inicilalizo el modelo
model = SVR()

#cross validation con gridsearch
gs = GridSearchCV(model,
                  tuned_parameters, 
                  cv=kfold,
                  scoring='neg_root_mean_squared_error')

#entreno el modelo
gs.fit(X_train, y_train)



GridSearchCV(cv=KFold(n_splits=5, random_state=7, shuffle=True),
             estimator=SVR(),
             param_grid=[{'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']},
                         {'C': [1, 10, 100, 1000], 'kernel': ['linear']}],
             scoring='neg_root_mean_squared_error')

In [25]:
#cuáles son los hiperparámetros óptimos?
print(gs.best_params_)

{'C': 1000, 'kernel': 'linear'}


In [26]:
#crear modelo usando parámetros óptimos
new_model_svr = SVR(kernel='linear', C=1000)

#Entreno modelo optimizado
new_model_svr.fit(X_train, y_train)

#Hacer el cross validation y probar con el train
r2_svr= (cross_val_score(new_model_tree, X_train,y_train, cv=kfold,  scoring='r2')).mean()
mae_svr= -(cross_val_score(new_model_tree, X_train,y_train, cv=kfold,  scoring='neg_mean_absolute_error')).mean()
rmse_svr= (-cross_val_score(new_model_tree, X_train,y_train, cv=kfold,  scoring='neg_root_mean_squared_error')).mean()

#Performance en el train
print("R2 train", r2_svr)
print("MAE Train", mae_svr)
print("MSE Train", rmse_svr)

R2 train 0.8833332245009938
MAE Train 2015.0780021702842
MSE Train 3019.23963196914


In [27]:
#Performance en el test
y_pred = new_model_svr.predict(X_test)

mae_svr=mean_absolute_error(y_test,y_pred)
rmse_svr=np.sqrt(mean_squared_error(y_test,y_pred))
r2_svr=r2_score(y_test,y_pred)

results_svr = pd.DataFrame([['SVR', mae_svr,rmse_svr,r2_svr]],columns=['Model', 'MAE', 'RMSE', 'R2'])
results_svr 

,Model,MAE,RMSE,R2
0,SVR,2814.094942,3724.912323,0.830836


#### Red Neuronal

Para este modelo vamos a usar Crossvalidation + GridSearch para encontrar los hiperparámetros óptimos:
- Tasa de aprendizaje
- Momentum



In [28]:
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.model_selection import cross_val_score
import tensorflow as tf

#creo función con estructura de la red neuronal
def create_model():
    # Neural network architecture
    model = Sequential()
    model.add(Dense(12,activation='tanh',input_dim=5))
    model.add(Dense(1,activation='linear'))
    
    return model

# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

# create model
model = KerasRegressor(model=create_model, loss='mean_squared_error', optimizer='SGD', epochs=200, batch_size=200, verbose=0)


# Definir los parámetros del grid search
learn_rate = [0.1, 0.05, 0.01]
momentum = [0.8, 0.6, 0.4]
param_grid = dict(optimizer__learning_rate=learn_rate, optimizer__momentum=momentum)

selection_score = make_scorer(mean_squared_error,greater_is_better=False) #MSE

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=2, scoring=selection_score)

#entreno modelo
grid_result = grid.fit(X_train, y_train)




In [29]:
#¿cuáles son los hiperparámetros óptimos?
print(grid_result.best_params_)

{'optimizer__learning_rate': 0.05, 'optimizer__momentum': 0.8}


In [30]:
#Diseñar modelo con métricas optimizadas

def create_model(lr=0.1,momentum=0.4):
    # Neural network architecture
    model = Sequential()
    model.add(Dense(12,activation='tanh',input_dim=5))
    model.add(Dense(1,activation='linear'))
    
    # Optimizer configuration
    opt = keras.optimizers.SGD(lr=lr,momentum=momentum)
    model.compile(loss = 'mean_squared_error',
                  optimizer=opt,
                  metrics=['mse'])
    return model

epochs = 200
batch_size = 100

model_search = KerasRegressor(build_fn=create_model,epochs=epochs)


#Entreno modelo optimizado
model_search.fit(X_train, y_train)

#Hacer el cross validation y probar con el train
r2_nn= (cross_val_score(model_search, X_train,y_train, cv=kfold,  scoring='r2')).mean()
mae_nn= -(cross_val_score(model_search, X_train,y_train, cv=kfold,  scoring='neg_mean_absolute_error')).mean()
mse_nn= (-cross_val_score(model_search, X_train,y_train, cv=kfold,  scoring='neg_root_mean_squared_error')).mean()

#Performance en el train
print("R2 train", r2_nn)
print("MAE Train", mae_nn)
print("RMSE Train", mse_nn)




C:\Users\Erick Machuca\anaconda3\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/200
22/22 [==============================] - 0s 620us/step - loss: 89875328.0000 - mse: 89875328.0000
Epoch 2/200
22/22 [==============================] - 0s 572us/step - loss: 29300378.0000 - mse: 29300378.0000
Epoch 3/200
22/22 [==============================] - 0s 620us/step - loss: 26326924.0000 - mse: 26326924.0000
Epoch 4/200
22/22 [==============================] - 0s 524us/step - loss: 22894144.0000 - mse: 22894144.0000
Epoch 5/200
22/22 [==============================] - 0s 620us/step - loss: 21950978.0000 - mse: 21950978.0000
Epoch 6/200
22/22 [==============================] - 0s 524us/step - loss: 20937466.0000 - mse: 20937466.0000
Epoch 7/200
22/22 [==============================] - 0s 572us/step - loss: 20484678.0000 - mse: 20484678.0000
Epoch 8/200
22/22 [==============================] - 0s 572us/step - loss: 20097656.0000 - mse: 20097656.0000
Epoch 9/200
22/22 [==============================] - 0s 572us/step - loss: 20042190.0000 - mse: 20042190.0000
Epoch 10/2

22/22 [==============================] - 0s 572us/step - loss: 19381350.0000 - mse: 19381350.0000
Epoch 75/200
22/22 [==============================] - 0s 572us/step - loss: 19472758.0000 - mse: 19472758.0000
Epoch 76/200
22/22 [==============================] - 0s 524us/step - loss: 19423528.0000 - mse: 19423528.0000
Epoch 77/200
22/22 [==============================] - 0s 572us/step - loss: 19390846.0000 - mse: 19390846.0000
Epoch 78/200
22/22 [==============================] - 0s 572us/step - loss: 19285082.0000 - mse: 19285082.0000
Epoch 79/200
22/22 [==============================] - 0s 572us/step - loss: 19513244.0000 - mse: 19513244.0000
Epoch 80/200
22/22 [==============================] - 0s 572us/step - loss: 19506010.0000 - mse: 19506010.0000
Epoch 81/200
22/22 [==============================] - 0s 572us/step - loss: 19583634.0000 - mse: 19583634.0000
Epoch 82/200
22/22 [==============================] - 0s 572us/step - loss: 19399146.0000 - mse: 19399146.0000
Epoch 83/200
2

22/22 [==============================] - 0s 572us/step - loss: 19336612.0000 - mse: 19336612.0000
Epoch 148/200
22/22 [==============================] - 0s 620us/step - loss: 19318914.0000 - mse: 19318914.0000
Epoch 149/200
22/22 [==============================] - 0s 572us/step - loss: 19274008.0000 - mse: 19274008.0000
Epoch 150/200
22/22 [==============================] - 0s 572us/step - loss: 19346586.0000 - mse: 19346586.0000
Epoch 151/200
22/22 [==============================] - 0s 572us/step - loss: 19588206.0000 - mse: 19588206.0000
Epoch 152/200
22/22 [==============================] - 0s 572us/step - loss: 19390876.0000 - mse: 19390876.0000
Epoch 153/200
22/22 [==============================] - 0s 572us/step - loss: 19382104.0000 - mse: 19382104.0000
Epoch 154/200
22/22 [==============================] - 0s 572us/step - loss: 19337712.0000 - mse: 19337712.0000
Epoch 155/200
22/22 [==============================] - 0s 572us/step - loss: 19399990.0000 - mse: 19399990.0000
Epoch 

C:\Users\Erick Machuca\anaconda3\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

17/17 [==============================] - 0s 1ms/step - loss: 19820646.0000 - mse: 19820646.0000
Epoch 149/200
17/17 [==============================] - 0s 1ms/step - loss: 19924168.0000 - mse: 19924168.0000
Epoch 150/200
17/17 [==============================] - 0s 1ms/step - loss: 19835434.0000 - mse: 19835434.0000
Epoch 151/200
17/17 [==============================] - 0s 813us/step - loss: 19857390.0000 - mse: 19857390.0000
Epoch 152/200
17/17 [==============================] - 0s 876us/step - loss: 19891924.0000 - mse: 19891924.0000
Epoch 153/200
17/17 [==============================] - 0s 1ms/step - loss: 19759134.0000 - mse: 19759134.0000
Epoch 154/200
17/17 [==============================] - 0s 1ms/step - loss: 20040186.0000 - mse: 20040186.0000
Epoch 155/200
17/17 [==============================] - 0s 813us/step - loss: 19826250.0000 - mse: 19826250.0000
Epoch 156/200
17/17 [==============================] - 0s 813us/step - loss: 19822786.0000 - mse: 19822786.0000
Epoch 157/200
17

C:\Users\Erick Machuca\anaconda3\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/200
17/17 [==============================] - 0s 876us/step - loss: 99906968.0000 - mse: 99906968.0000
Epoch 2/200
17/17 [==============================] - 0s 813us/step - loss: 35138136.0000 - mse: 35138136.0000
Epoch 3/200
17/17 [==============================] - 0s 876us/step - loss: 26917814.0000 - mse: 26917814.0000
Epoch 4/200
17/17 [==============================] - 0s 813us/step - loss: 25292088.0000 - mse: 25292088.0000
Epoch 5/200
17/17 [==============================] - 0s 1ms/step - loss: 26029852.0000 - mse: 26029852.0000
Epoch 6/200
17/17 [==============================] - 0s 813us/step - loss: 26474478.0000 - mse: 26474478.0000
Epoch 7/200
17/17 [==============================] - 0s 813us/step - loss: 25224120.0000 - mse: 25224120.0000
Epoch 8/200
17/17 [==============================] - 0s 813us/step - loss: 24702780.0000 - mse: 24702780.0000
Epoch 9/200
17/17 [==============================] - 0s 751us/step - loss: 24878784.0000 - mse: 24878784.0000
Epoch 10/200

17/17 [==============================] - 0s 1ms/step - loss: 21172072.0000 - mse: 21172072.0000
Epoch 76/200
17/17 [==============================] - 0s 938us/step - loss: 21130102.0000 - mse: 21130102.0000
Epoch 77/200
17/17 [==============================] - 0s 813us/step - loss: 21174170.0000 - mse: 21174170.0000
Epoch 78/200
17/17 [==============================] - 0s 1ms/step - loss: 21106708.0000 - mse: 21106708.0000
Epoch 79/200
17/17 [==============================] - 0s 1ms/step - loss: 21198448.0000 - mse: 21198448.0000
Epoch 80/200
17/17 [==============================] - 0s 813us/step - loss: 21104172.0000 - mse: 21104172.0000
Epoch 81/200
17/17 [==============================] - 0s 813us/step - loss: 21218210.0000 - mse: 21218210.0000
Epoch 82/200
17/17 [==============================] - 0s 813us/step - loss: 21132896.0000 - mse: 21132896.0000
Epoch 83/200
17/17 [==============================] - 0s 813us/step - loss: 21108978.0000 - mse: 21108978.0000
Epoch 84/200
17/17 [

17/17 [==============================] - 0s 813us/step - loss: 21153648.0000 - mse: 21153648.0000
Epoch 150/200
17/17 [==============================] - 0s 813us/step - loss: 21078354.0000 - mse: 21078354.0000
Epoch 151/200
17/17 [==============================] - 0s 876us/step - loss: 21260870.0000 - mse: 21260870.0000
Epoch 152/200
17/17 [==============================] - 0s 877us/step - loss: 21160670.0000 - mse: 21160670.0000
Epoch 153/200
17/17 [==============================] - 0s 814us/step - loss: 21082366.0000 - mse: 21082366.0000
Epoch 154/200
17/17 [==============================] - 0s 941us/step - loss: 21136448.0000 - mse: 21136448.0000
Epoch 155/200
17/17 [==============================] - 0s 877us/step - loss: 21143304.0000 - mse: 21143304.0000
Epoch 156/200
17/17 [==============================] - 0s 877us/step - loss: 21205110.0000 - mse: 21205110.0000
Epoch 157/200
17/17 [==============================] - 0s 815us/step - loss: 21182714.0000 - mse: 21182714.0000
Epoch 

In [31]:
#Performance en el test
y_pred = model_search.predict(X_test)

mae_nn=mean_absolute_error(y_test,y_pred)
mse_nn=np.sqrt(mean_squared_error(y_test,y_pred))
r2_nn=r2_score(y_test,y_pred)

results_nn = pd.DataFrame([['Neural Network', mae_nn,mse_nn,r2_nn]],columns=['Model', 'MAE', 'RMSE', 'R2'])
results_nn 

10/10 [==============================] - 0s 1ms/step


,Model,MAE,RMSE,R2
0,Neural Network,3341.397303,4338.440975,0.770521


### Comparación de modelos

In [32]:
data_compar_cross = {'Modelo': ['LR', 'Ridge', 'RF','Tree', ' SVR', 'NN'], 'RMSE': [rmse_lr, rmse_ridge, rmse_rf, rmse_tree,rmse_svr, mse_nn],
                    'MAE':[mae_lr, mae_ridge, mae_rf, mae_tree,mae_svr, mae_nn],
                    'R2':[r2_lr, r2_ridge, r2_rf, r2_tree,r2_svr, r2_nn]} 
data_compar_cross = pd.DataFrame(data_compar_cross)
data_compar_cross

,Modelo,RMSE,MAE,R2
0,LR,3666.939818,2842.934786,0.836061
1,Ridge,3666.979515,2844.436399,0.836057
2,RF,2561.377387,1836.106102,0.920012
3,Tree,2880.276684,1960.984898,0.898855
4,SVR,3724.912323,2814.094942,0.830836
5,NN,4338.440975,3341.397303,0.770521


Para estos datos, el mejor modelo el el Random Forest (Bosque Aleatorio). 

Por lo tanto utilizaremos este modelo para predecir datos nuevos

In [33]:
#crear nuevos datos
X_new  = pd.DataFrame([[27, 0, 30, 5999, 7000]], columns=['age', 'gender', 'miles', 'debt', 'income'])
X_new

,age,gender,miles,debt,income
0,27,0,30,5999,7000


In [34]:
# ¿Cuál sería el valor del carro?

ynew = new_model_RF.predict(X_new)



C:\Users\Erick Machuca\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [35]:
print('el valor del carro sería', ynew)


el valor del carro sería [28539.81413163]
